## siddhant (resnet)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
data_path = '/content/drive/MyDrive/IPD/code/image/hate_symbols/data'
train_path = '/content/drive/MyDrive/IPD/code/image/hate_symbols/train'
valid_path = '/content/drive/MyDrive/IPD/code/image/hate_symbols/valid'

In [ ]:
# for dir in os.listdir(data_path):
#   new_folder_path = os.path.join(train_path, dir)
#   new_folder_path = os.path.join(valid_path, dir)
#   os.makedirs(new_folder_path, exist_ok=True)

In [ ]:
# from PIL import Image
# import os

# for dir in os.listdir(data_path):
#   new_folder_path = os.path.join(train_path, dir)
#   for file in os.listdir(os.path.join(data_path,dir)):
#     try:
#       img = Image.open(os.path.join(data_path,dir,file))
#       img = img.resize((224,224))
#       rgb_img = img.convert('RGB')
#       rgb_img.save(os.path.join(new_folder_path,file.split('.')[0]+'.jpg'))
#     except:
#       pass

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

num_classes = 5
image_size = (224, 224)
batch_size = 8
epochs = 10

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=image_size + (3,))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

steps_per_epoch = len(train_generator)//batch_size
validation_steps = len(val_generator)//batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Found 1043 images belonging to 5 classes.
Found 258 images belonging to 5 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 182s 7s/step - accuracy: 0.3727 - loss: 3.1876 - val_accuracy: 0.6562 - val_loss: 268.1530
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 111s 7s/step - accuracy: 0.6215 - loss: 1.8805 - val_accuracy: 0.0625 - val_loss: 1213326.2500
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 114s 7s/step - accuracy: 0.5550 - loss: 1.8705 - val_accuracy: 0.5938 - val_loss: 3223.9243
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 113s 7s/step - accuracy: 0.7531 - loss: 1.0417 - val_accuracy: 0.3125 - val_loss: 130.9483
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 109s 7s/step - accuracy: 0.6531 - loss: 1.1767 - val_accuracy: 0.6562 - val_loss: 36.2811
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 107s 7s/step - accuracy: 0.7374 - loss: 0.9580 - val_accuracy: 0.6875 - val_loss: 4.0457
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 108s 7s/step - accuracy: 0.7369 - loss: 1.0435 - val_accuracy: 0.8125 - val_loss: 3.2875
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 107s 7s/step - accuracy: 0.7420 - loss: 0.9411 - val_accuracy: 0.8125 

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 916ms/step - accuracy: 0.6654 - loss: 1.0344 - val_accuracy: 1.0000 - val_loss: 0.8995
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 175s 7s/step - accuracy: 0.6649 - loss: 1.2245 - val_accuracy: 0.0625 - val_loss: 13.7532


In [ ]:
model.save('symbol_detector_resnet.keras')

In [ ]:
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}
print(index_to_class)

{0: 'cut_throat_gesture', 1: 'finger_gun_to_the_head', 2: 'middle_finger', 3: 'slanted_eyes_gesture', 4: 'swastika'}


In [ ]:
def predict_single_image(image_path, model, image_size):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    print(predictions)
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class, predictions

image_path = '/content/swastika.png'
predicted_class, predictions = predict_single_image(image_path, model, image_size)
print(f"Predicted Class: {predicted_class}")
print(f"Predictions: {predictions}")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
[[3.5266859e-11 2.4311834e-03 9.9756879e-01 1.9456777e-16 6.6996819e-17]]
Predicted Class: [2]
Predictions: [[3.5266859e-11 2.4311834e-03 9.9756879e-01 1.9456777e-16 6.6996819e-17]]


## tirath (ViT & CLIP) (no finetuning)

In [ ]:
import torch
from transformers import ViTForImageClassification, ViTImageProcessor
from torchvision import transforms
from PIL import Image

In [ ]:
class_names = ['cut_throat_gesture', 'finger_gun_to_the_head', 'middle_finger', 'slanted_eyes_gesture', 'swastika', 'not hate']

In [ ]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=len(class_names), ignore_mismatched_sizes=True)
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

In [ ]:
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(image)

    predicted_class_idx = outputs.logits.argmax(-1).item()
    return class_names[predicted_class_idx]

In [ ]:
image_path = '/content/car.jpg'
predicted_class = predict_image(image_path)
print(f'Predicted class: {predicted_class}')

Predicted class: cut_throat_gesture


## tirath (ViT) (finetuning)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, AdamW, get_scheduler

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/IPD/code/image/hate_symbols/train/', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=5,
    ignore_mismatched_sizes=True
).to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 10
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images).logits
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

Epoch [1/10], Loss: 0.7773437485462282
Epoch [2/10], Loss: 0.18906403496498014
Epoch [3/10], Loss: 0.053430866586362445
Epoch [4/10], Loss: 0.022554007631431266
Epoch [5/10], Loss: 0.013084341567463992
Epoch [6/10], Loss: 0.009313693360947981
Epoch [7/10], Loss: 0.0073364293202757835
Epoch [8/10], Loss: 0.006213190244147327
Epoch [9/10], Loss: 0.0055979902333602675
Epoch [10/10], Loss: 0.005284533114740398


In [ ]:
model.save_pretrained('vit-hateful-gesture-classification')

In [ ]:
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [ ]:
class_to_idx = train_dataset.class_to_idx
class_names = list(class_to_idx.keys())
print(class_names)

['cut_throat_gesture', 'finger_gun_to_the_head', 'middle_finger', 'slanted_eyes_gesture', 'swastika']


In [ ]:
from PIL import Image
import torch
from torchvision import transforms

def predict_class(image_path):
    image = Image.open(image_path).convert('RGB')
    image = data_transforms(image).unsqueeze(0)
    image = image.to(device)
    with torch.no_grad():
        outputs = model(image).logits
    predicted_class_idx = outputs.argmax(-1).item()
    predicted_class = class_names[predicted_class_idx]
    return predicted_class

In [ ]:
image_path = "/content/swastika.png"

In [ ]:
print(f"Prediction: {predict_class(image_path)}")

Prediction: swastika


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


## Creating HF space for model

In [ ]:
!pip install gradio==3.14.0 pydantic==1.10.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 558.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
import gradio as gr

In [ ]:
model = ViTForImageClassification.from_pretrained('Tirath5504/IPD-Image-ViT-Finetune')
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
class_names = ['cut_throat_gesture', 'finger_gun_to_the_head', 'middle_finger', 'slanted_eyes_gesture', 'swastika']

In [ ]:
def predict(image):
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs).logits

    predicted_class_idx = outputs.argmax(-1).item()
    predicted_class = class_names[predicted_class_idx]

    return predicted_class

In [ ]:
iface = gr.Interface(fn=predict,
                     inputs=gr.Image(type="pil"),
                     outputs=gr.Label(num_top_classes=1),
                     title="Hateful Content Detection",
                     description="Upload an image to classify hateful gestures or symbols")

IMPORTANT: You are using gradio version 3.14.0, however version 4.29.0 is available, please upgrade.
--------


In [ ]:
iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://ee345043-be98-40f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
